# PROJET SEATTLE ENERGY BENCHMARKING
## Notebook 01 : Évaluation  de la qualité des données

---

### Identité du document
* **Statut :** Phase 1 (exploration & prototypage)
* **Date de création :** 29 Décembre 2025
* **Dernière mise à jour :** 29 Décembre 2025
* **Dépendances notebooks** : Notebook 0

### Description
Ce notebook constitue un **audit de qualité systématique**. Nous adoptons ici une posture : méthodique, exhaustive et sans présupposés. 
L'objectif n'est pas encore de modifier les données, mais de détecter, quantifier et documenter tous les problèmes (valeurs manquantes, aberrantes, incohérentes) afin de définir une stratégie de nettoyage factuelle.

> **Philosophie :** "Mieux vaut détecter les problèmes maintenant que découvrir qu'on a analysé des données biaisées après 2 semaines de travail."

### Objectifs principaux
1. **Cartographier** la complétude des données (analyse des NaN).
2. **Distinguer** les erreurs manifestes des valeurs extrêmes légitimes (Outliers).
3. **Valider** la cohérence interne (logique métier et physique).
4. **Évaluer** l'impact du flag `DefaultData`.
5. **Produire** un plan d'action pour le nettoyage.

### Dépendances critiques
* `hydra` : Gestion de configuration.
* `pandas`, `seaborn`, `matplotlib` : Manipulation et inspection visuelle.
* `src.data_quality/` : Fonctions à construire.

### LIVRABLES
1. **Rapport de Qualité**  : Synthèse des constats.
2. **Matrice de Stratégie**  : Plan d'action ligne par ligne.
3. **Figures d'audit** : Preuves visuelles.

---

---
# Table des Matières
- [Section 0 : Importation et chargement](#section-0--importation-et-chargement)
- [Section 1 : Où sont les données manquantes et pourquoi ? (Completeness)](#section-1--où-sont-les-données-manquantes-et-pourquoi)
- [Section 2 : Les valeurs extrêmes sont-elles des erreurs ou des réalités ? (Outliers)](#section-2--les-valeurs-extrêmes-sont-elles-des-erreurs-ou-des-réalités)
- [Section 3 : Les données respectent-elles la logique physique et métier ? (Consistency)](#section-3--les-données-respectent-elles-la-logique-physique-et-métier)
- [Section 4 : Le flag "DefaultData" compromet-il la fiabilité ? (Reliability)](#section-4--le-flag-defaultdata-compromet-il-la-fiabilité)
- [Section 5 : Existe-t-il des doublons ou des quasi-doublons ? (Uniqueness)](#section-5--existe-t-il-des-doublons-ou-des-quasi-doublons)
- [Section 6 : Quelle stratégie de nettoyage adopter ? (Action Plan)](#section-6--quelle-stratégie-de-nettoyage-adopter)

---

# Section 0 : Importation et chargement

## Importation des packages

In [1]:
import logging
import pandas as pd
from pathlib import Path


# Import des fonctions utilitaires 

import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
SRC_PATH = PROJECT_ROOT / "src"

if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))


from data.load_data import load_data_raw
from utils.config_loader import load_config, create_directories
from utils.eda_logger import setup_eda_logger
 


## init config et chargement

In [2]:
# Chargement de la configuration principale (Hydra)
cfg = load_config()

# Initialisation du logger
setup_eda_logger(cfg)
logger = logging.getLogger(__name__)

# Création des dossiers nécessaires à l'exécution (raw, interim, processed, reports, etc.)
create_directories(cfg)
# Chargement des données brutes
df_raw = load_data_raw(cfg)

# Validation dimensions
n_rows, n_cols = df_raw.shape
logger.info(f"Dataset chargé ({n_rows} lignes, {n_cols} colonnes)")

# Premier aperçu
df_raw.sample(5)


2025-12-29 22:26:22,645 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\raw
2025-12-29 22:26:22,646 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\interim
2025-12-29 22:26:22,648 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\processed
2025-12-29 22:26:22,649 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\figures
2025-12-29 22:26:22,651 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\reports
2025-12-29 22:26:22,652 - data.load_data - INFO - Fichier local détecté : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\raw\2016_Building_Energy_Benchmarking.csv
2025-12-29 22:26:22,681 - data.load_data - INFO - ✓ DataFrame chargé : 3376 lignes,

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.0,0659000970,...,1.573449e+06,5368607.0,88039.984380,8803998.0,False,NaN,Compliant,NaN,505.01,2.88
494,673,2016,NonResidential,Retail Store,#8944 West Seattle,7345 Delridge Way SW,Seattle,WA,98106.0,7985400295,...,1.225938e+06,4182901.0,12346.966800,1234697.0,False,NaN,Compliant,NaN,94.74,1.00
658,19481,2016,NonResidential,Distribution Center,Shultz Distribution,6801 EAST MARGINAL WAY S,Seattle,WA,98108.0,0001800159,...,2.580906e+05,880605.0,3435.809814,343581.0,False,NaN,Compliant,NaN,24.39,0.88
145,233,2016,Nonresidential COS,Other,Armory Building,860 Terry Ave N,Seattle,WA,98109.0,4088803210,...,1.113108e+06,3797924.0,37508.820310,3750882.0,False,NaN,Compliant,NaN,225.69,4.51
1891,23713,2016,Multifamily LR (1-4),Low-Rise Multifamily,Starliter,8708 Rainier Ave South,Seattle,WA,98118.0,3524049011,...,2.295147e+05,783104.0,0.000000,0.0,False,NaN,Compliant,NaN,5.46,0.25


## Section 1 : Où sont les données manquantes et pourquoi ?

**Objectif :** Produire une vue complète et hiérarchisée de la "missingness" pour distinguer les absences aléatoires des absences structurelles.

**Questions clés :**
* Quelles colonnes dépassent le seuil critique de 50% de NaN ?
* Existe-t-il une corrélation entre les absences (ex: si A est vide, B est-elle toujours vide ?) ?
* Classification du mécanisme : MCAR (Aléatoire complet), MAR (Aléatoire lié), ou MNAR (Non aléatoire/Biais) ?

In [ ]:
# code

## Section 2 : Les valeurs extrêmes sont-elles des erreurs ou des réalités ?